In [1]:

import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import cv2
import random 
import os, os.path
import matplotlib.pyplot as plt                                     
import imutils                                                      #contours

from torch.utils.data import Dataset, DataLoader, random_split
from torchinfo import summary                                       #network description 
from sympy import *                                                 #symbolic mathematics


In [18]:
class Student(nn.Module):
  def __init__(self, input_size, num_classes):
    super(Student, self).__init__()
    self.fc1 = nn.Linear(input_size, 30)
    self.fc2 = nn.Linear(30, num_classes)

  def forward(self, x):
    x = x.view(-1, 784)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    return x


model = Student(input_size=784, num_classes=18)
model.load_state_dict(torch.load("model_new_student.pth"))


#get parameters 
for name, params in model.named_parameters()
  np.savetxt(name + ".txt", params.detach().numpy(), fmt="%s")




784


TypeError: len() of a 0-d tensor